# Teste de arquitetura de ConvNet com múltiplas entradas em Pytorch

In [1]:
import pandas as pd

from torch.utils.data import Dataset, DataLoader
import torchvision as tvis
from torch import nn
import torch

from loguru import logger

/home/itsmeale/code/face-verification/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
BAD_IMAGES = [
    'Abdullah_al-Attiyah_0001.jpg',
    'Dereck_Whittenburg_0001.jpg',
    'Lawrence_Foley_0001.jpg'
]

In [23]:
class LFWDataset(Dataset):
    
    def __init__(self, path: str, preprocessed_prefix: str):
        self.data = list()
        self.path = path
        self.preprocessed_prefix = preprocessed_prefix
        self.load_df()
        self.load_images()

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # returns x1, x2, y
        return self.data[idx]

    def filter_bad_images(self, df):
        return df[
            ~df.img1_full_id.isin(BAD_IMAGES) &
            ~df.img2_full_id.isin(BAD_IMAGES)
        ] 
    
    def load_df(self):
        logger.info("Loading pairs dataframe")
        df = pd.read_parquet(self.path).pipe(self.filter_bad_images)
        df["prepro_img1"] = df.img1_full_id.apply(lambda s: f"{self.preprocessed_prefix}/{s}")
        df["prepro_img2"] = df.img2_full_id.apply(lambda s: f"{self.preprocessed_prefix}/{s}")
        self.data_tuples = df[["prepro_img1", "prepro_img2", "match"]].values.copy()
        del df
    
    def load_images(self):
        logger.info("Loading images")
        for im1_path, im2_path, match in self.data_tuples:
            im1 = tvis.io.read_image(im1_path).float().to(DEVICE)
            im2 = tvis.io.read_image(im2_path).float().to(DEVICE)
            self.data.append((im1, im2, torch.as_tensor(match).float().to(DEVICE)))

In [24]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=1,
            kernel_size=5
        )
        self.pool1 = nn.AvgPool2d(10)
        self.fc = nn.Linear(
            in_features=196*2,
            out_features=1,
        )
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
    
    def make_conv(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool1(x)
        return torch.flatten(x, 1)
    
    def forward(self, x1, x2):
        x1 = self.make_conv(x1)
        x2 = self.make_conv(x2)
        x = torch.concat([x1, x2], dim=1)
        x = self.fc(x)
        y = self.softmax(x)
        return y

In [36]:
def train_loop(loader, model, loss_fn, optimizer):
    size = len(loader.dataset)
    for batch, (x1, x2, y) in enumerate(loader):
        pred = model.forward(x1, x2)
        loss = loss_fn(pred, y.unsqueeze(1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return loss

In [34]:
dataset = LFWDataset(
    path="../data/interim/pairs.parquet",
    preprocessed_prefix="../data/preprocessed/images"
)
loader = DataLoader(dataset, batch_size=64, shuffle=True)
model = ConvNet()
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
epochs = 10

2022-12-28 19:39:14.766 | INFO     | __main__:load_df:24 - Loading pairs dataframe
2022-12-28 19:39:14.784 | INFO     | __main__:load_images:32 - Loading images


In [35]:
model = model.to(DEVICE)

In [37]:
for epoch in range(epochs):
    loss = train_loop(loader, model, loss_fn, optimizer)
    print(f"epoch {epoch} - loss: {loss:>7f}")

epoch 0 - loss: 1.899425
epoch 1 - loss: 2.362541
epoch 2 - loss: 2.433667
epoch 3 - loss: 2.496488
epoch 4 - loss: 1.897906
epoch 5 - loss: 2.435255
epoch 6 - loss: 1.962169
epoch 7 - loss: 1.898244
epoch 8 - loss: 1.493037
epoch 9 - loss: 1.699473
